# Obtenção da base de dados

## 1. Instalção e importe de bibliotecas úteis

In [ ]:
%pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [2]:
import os
import cv2
import time
import tensorboard
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

## 2. Obtenção dos Pontos de Articulação por método Holístico pelo Mediapipe

In [3]:
mp_holistic = mp.solutions.holistic # Modelo holístico
mp_drawing = mp.solutions.drawing_utils # Desenhando o modelo

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Conversão das cores de Azul, Verde, Vermelho para Vermelho, Verde, Azul
    image.flags.writeable = False                  # Imagem não é mais gravável
    results = model.process(image)                 # Faz a predição da imagem
    image.flags.writeable = True                   # Imagem torna-se gravável
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Conversão das cores de Vermelho, Verde, Azul para Azul, Verde, Vermelho
    return image, results

In [5]:
def draw_landmarks(image, results):
    # Desenhando as conexões da rosto
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            )
    # Desenhando as conexões da postura
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=3),
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            )
    # Desenhando as conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=3),
                            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            )  
    # Desenhando as conexões da mão direita
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=3),
                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                            ) 

In [9]:
cap = cv2.VideoCapture(0)
# Definindo o modelo do Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Lendo o retorno da chamada
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)

        # Desenhando landmarks
        draw_landmarks(image, results)

        # Mostrando na tela
        cv2.imshow('OpenCV feed', image)

        # Desligando a tela sem quebrar o código (aperte 'q para fechar')
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

## 3. Obtendo valor dos Pontos de Articulação

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, right_hand, left_hand])

## 4. Definindo pastas de coleta de dados

In [7]:
# Caminho para exportar os dados dos arrays do numpy
DATA_PATH = os.path.join('MP_Data')

# Sinais a serem detectados
actions = np.array(['OBRIGADO', 'OI']) #, 'DESCULPAR', 'POR-FAVOR', 'ENTENDER'])

# Trinta videos a serem gravados
no_sequences = 20

# Os videos terão dez frames de duração
sequence_length = 22

In [8]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Coletando Pontos de Articulação para teste e treinamento

In [8]:
cap = cv2.VideoCapture(0)
# Definindo o modelo do Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:

    # Repetição dos sinais
    for action in actions:
        # Repetição da quantidade de videos
        for sequence in range(no_sequences):
            # Repetição dos quadros a serem capturados
            for frame_num in range(sequence_length + 2):
                # Lendo o retorno da chamada
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                # Desenhando landmarks
                draw_landmarks(image, results)

                # Lógica da sequência de abertura da câmera
                if frame_num == 0 and sequence == 0:
                    cv2.putText(image, 'INICIANDO COLETA DO SINAL {}'.format(action), (10, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
                    # Abrindo a tela
                    cv2.imshow('Tela OpenCV', image)
                    cv2.waitKey(5000)

                elif frame_num == 1:
                    cv2.putText(image, 'INICIANDO VIDEO NUMERO {}'.format(sequence + 1), (10, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
                    # Abrindo a tela
                    cv2.imshow('Tela OpenCV', image)
                    cv2.waitKey(0)

                elif frame_num > 1:
                    cv2.putText(image, 'Coletando quadros para {} Video numero {}'.format(action, sequence + 1), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    
                    # Abrindo a tela
                    cv2.imshow('Tela OpenCV', image)
                    # Exportando Pontos de Articulação
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num-2))
                    np.save(npy_path, keypoints)

                # Desligando a tela sem quebrar o código (aperte 'q para fechar')
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

## 6. Preprocessamento dos dados e criação das ferramentas e rótulos

In [9]:
#array de objetos classificadores
label_map = {label: num for num, label in enumerate(actions)}

In [11]:
#sequenciamento dos pontos de articulação e suas respectivas classificações
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            if(frame_num > 1):
                # res = []
                # for i in teste.replace('[','').replace(']','').split(', '):
                #     res.append(float(i))
                res = np.load(os.path.join(DATA_PATH, action, str(sequence), '{}.npy'.format(frame_num)))
                window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

print(labels)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# Processo de Teste

## 7. Separação dos dados para treino/teste

In [16]:
#Definindo dados para treino
X = np.array(sequences)

In [17]:
#Definindo labels para treino
Y = to_categorical(labels).astype(int)

In [20]:
#Dividindo dados para treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25)

In [21]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(15, 18, 1629) (5, 18, 1629) (15, 2) (5, 2)


## 8. Construindo e treinando uma Rede Neural LSTM

In [22]:
#Salvando log do TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
#Configurando o modelo de previsão
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20,1629)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
#Treinando o modelo
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, Y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
1/1 [==============================] - 14s 14s/step - loss: 0.6901 - categorical_accuracy: 0.5333
Epoch 2/1000
1/1 [==============================] - 1s 606ms/step - loss: 1.1419 - categorical_accuracy: 0.5333
Epoch 3/1000
1/1 [==============================] - 0s 85ms/step - loss: 0.7310 - categorical_accuracy: 0.4000
Epoch 4/1000
1/1 [==============================] - 0s 87ms/step - loss: 2.5649 - categorical_accuracy: 0.4667
Epoch 5/1000
1/1 [==============================] - 0s 81ms/step - loss: 1.0076 - categorical_accuracy: 0.5333
Epoch 6/1000
1/1 [==============================] - 0s 92ms/step - loss: 0.9104 - categorical_accuracy: 0.4667
Epoch 7/1000
1/1 [==============================] - 0s 108ms/step - loss: 0.7493 - categorical_accuracy: 0.5333
Epoch 8/1000
1/1 [==============================] - 0s 121ms/step - loss: 0.6894 - categorical_accuracy: 0.5333
Epoch 9/1000
1/1 [==============================] - 0s 130ms/step - loss: 0.7396 - categorical_accuracy: 0.46

In [10]:
%reload_ext tensorboard

In [11]:
%tensorboard --logdir Logs

Reusing TensorBoard on port 6006 (pid 15660), started 4 days, 15:35:43 ago. (Use '!kill 15660' to kill it.)

## 9. Validando o modelo usando Matriz de Confusão e Acuracidade

In [1]:
#Processo de teste do modelo
yhat = model.predict(X_train)

NameError: name 'model' is not defined

In [ ]:
#Matriz de confusão
ytrue = np.argmax(Y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
#Acurácia do modelo
accuracy_score(ytrue, yhat)

# Em Progresso

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0,85+num*40),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

In [ ]:
# 1. Novas variáveis de detecção
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Definindo o modelo do Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Lendo o retorno da chamada
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)

        # Desenhando landmarks
        draw_landmarks(image, results)
        
        # 2. lógica de previsão
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))

        # 3. lógica de visualização
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Mostrando na tela
        cv2.imshow('OpenCV feed', image)

        # Desligando a tela sem quebrar o código (aperte 'q para fechar')
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()